# Edge generation 

Generates the edges as a dataframe

Load libraries

In [1]:
modules = ["pandas", "pprintpp","rich","tf_keras", "sentence-transformers", "ipywidgets"]

import sys
import os.path
from subprocess import check_call
import importlib
import os

def instala(modules):
  print("Instalando módulos")
  for m in modules:
      # para el import quitamos [...] y ==...
      p = m.find("[")
      mi = m if p==-1 else m[:p]
      p = mi.find("==")
      mi = mi if p==-1 else mi[:p]

      torch_loader = importlib.util.find_spec(mi)
      if torch_loader is not None:
          print(m," encontrado")
      else:
          print(m," No encontrado, instalando...",end="")  
          try:        
            r = check_call([sys.executable, "-m", "pip", "install", "--user", m])
            print("¡hecho!")
          except:
            print("¡Problema al instalar ",m,"! ¿seguro que el módulo existe?",sep="")
              
  print("¡Terminado!")

instala(modules) 

Instalando módulos
pandas  encontrado
pprintpp  encontrado
rich  encontrado
tf_keras  encontrado
sentence-transformers  No encontrado, instalando...¡hecho!
ipywidgets  encontrado
¡Terminado!


Load data

In [ ]:
import pandas as pd
pd.reset_option('display.max_colwidth')

def extract_from_user(id):
    year = int(id[-5:-3])
    party = id[-3:]
    user = id[:-6]
    return user,year, party
    
user = "usa162024.csv"
encoding = "encoding_user_768.csv"
import pandas as pd
df_user = pd.read_csv(user)
df_encoding = pd.read_csv(encoding)

import json
import numpy as np
df_encoding["embedding"] = df_encoding["embedding"].apply(json.loads)
df_encoding["embedding"] = df_encoding["embedding"].apply(lambda x: np.array(x, dtype=float))
# extract components from user
df_encoding["UserOri"] = df_encoding["User"].apply(lambda x: extract_from_user(x)[0])
df_encoding["year"] = df_encoding["User"].apply(lambda x: extract_from_user(x)[1])
df_encoding["party"] = df_encoding["User"].apply(lambda x: extract_from_user(x)[2])

df_encoding

,User,embedding,UserOri,year,party
0,1111Bill_16dem,"[-0.06382136046886444, 0.1339360475540161, -0....",1111Bill,16,dem
1,1111Bill_20dem,"[0.21235774457454681, 0.14392971992492676, -0....",1111Bill,20,dem
2,1111Bill_20rep,"[-0.2249380201101303, -0.1867368072271347, -0....",1111Bill,20,rep
3,1111Bill_24rep,"[-0.19874753057956696, -0.43008577823638916, -...",1111Bill,24,rep
4,12MileGeo_16rep,"[-0.031242497265338898, -0.18871095776557922, ...",12MileGeo,16,rep
...,...,...,...,...,...
11116,zulran_24rep,"[-0.2998636066913605, -0.21201753616333008, -0...",zulran,24,rep
11117,zzelda01_16dem,"[-0.2710208296775818, 0.10389921069145203, -0....",zzelda01,16,dem
11118,zzelda01_16rep,"[-0.015369012951850891, 0.28082552552223206, -...",zzelda01,16,rep
11119,zzelda01_20rep,"[-0.21957272291183472, 0.059779584407806396, -...",zzelda01,20,rep


In [4]:
df_user

,Unnamed: 0,User,Tweet,Date,year,party
0,0,GreggStallings,RT @seanspicer: Not that they were ever that h...,2016-11-02,16,dem
1,1,GreggStallings,RT @CaliforniaPanda: SHAMEFUL! Vietnam vet boo...,2016-11-03,16,dem
2,2,GreggStallings,RT @KellyannePolls: No wonder big majorities w...,2016-11-03,16,dem
3,3,psbaloans4u,@HillaryClinton you madam secretary are the da...,2016-11-02,16,dem
4,4,psbaloans4u,We have @POTUS and @HillaryClinton for the Ira...,2016-11-03,16,dem
...,...,...,...,...,...,...
63605,63605,January_Frosts,This wins HaIIoween !! Hands down best perform...,2024-11-01,24,rep
63606,63606,meredithheron,INBOX: @DA_LarryKrasner responds to @realdonal...,2024-11-05,24,rep
63607,63607,shirleybedsole,Almost everyone I know serving in the US milit...,2024-10-31,24,rep
63608,63608,rcsutton,half empty venues for @realDonaldTrump in PA,2024-11-04,24,rep


In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cosine

def extract_from_user(id):
    year = int(id[-5:-3])
    party = id[-3:]
    user = id[:-6]
    return user,year, party
    
def similar_users(df, user_id):
    k = 0.85
    # Obtain the embedding of the user
    user_vector = df.loc[df['User'] == user_id, 'embedding'].values

    if len(user_vector) == 0:
        raise ValueError(f"El usuario {user_id} no está en el DataFrame.")
    
    user_vector = user_vector[0]  # we will compare with this codificacion

    # Calculate the cosine similarity with all other users
    similitudes = []
    
    for _, row in df.iterrows():
        if row['User'] != user_id:  # Avoid comparing the user with itself
            #print(len(user_vector), len(row['embedding']))
            sim = 1 - cosine(user_vector, row['embedding'])  # 1 - cosine distance = cosine similarity
            similitudes.append((row['User'], sim))

    # sort by similarity from highest to lowest and select those above k
    similitudes = sorted(similitudes, key=lambda x: x[1], reverse=True)
    usuariosSimilares = []
    grado = []
    i=0
    while i<len(similitudes) and similitudes[i][1]>=k:
        usuariosSimilares.append(similitudes[i][0])
        grado.append((similitudes[i][1]-k)/(1-k)) # between 0 and 1
        i+=1

    # Convert to DataFrame
    df_similares = pd.DataFrame({"user1":[user_id]*len(grado), "user2":usuariosSimilares, "weight":grado})

    return df_similares

In [9]:
from tqdm import tqdm

i =0
user = []
degree = []
for year in [16,20,24]:
    for party in ["dem","rep"]:
        df_restricted = df_encoding[(df_encoding.year == year) & (df_encoding.party==party)]
        print(year,party)
        for _, user_id in tqdm(enumerate(df_restricted["User"])):
            df_one =  similar_users(df_restricted, user_id)
            if i==0:
                df_graph = df_one
            else:
                df_graph = pd.concat([df_graph,df_one])
            user.append(user_id)
            degree.append(len(df_one))
            i+=1


16 dem


2024it [01:32, 22.00it/s]


16 rep


2079it [01:42, 20.27it/s]


20 dem


...

2199it [01:57, 18.77it/s]


24 dem


1674it [01:10, 23.86it/s]


24 rep


1521it [00:57, 26.36it/s]


In [11]:
len(user)

11121

In [13]:
df_graph

,user1,user2,weight
0,1111Bill_16dem,Indianaborn_16dem,1.000000
0,1Cubbie_16dem,janne_clark_16dem,0.713589
1,1Cubbie_16dem,cisppetta_16dem,0.709196
2,1Cubbie_16dem,Zxtoo_16dem,0.703619
3,1Cubbie_16dem,edzsplace_16dem,0.699791
...,...,...,...
1,zulran_24rep,cyndigist_24rep,1.000000
2,zulran_24rep,dd9000_24rep,1.000000
3,zulran_24rep,eliza_shines_24rep,1.000000
4,zulran_24rep,islandinthemoon_24rep,1.000000


In [15]:
df_graph["party"] = df_graph["user1"].apply(lambda x: (x[-3:]))
df_graph["Y"] = df_graph["user1"].apply(lambda x: int(x[-5:-3]))
df_graph["user1"] = df_graph["user1"].apply(lambda x: (x[:-6]))
df_graph["user2"] = df_graph["user2"].apply(lambda x: (x[:-6]))
df_graph

,user1,user2,weight,party,Y
0,1111Bill,Indianaborn,1.000000,dem,16
0,1Cubbie,janne_clark,0.713589,dem,16
1,1Cubbie,cisppetta,0.709196,dem,16
2,1Cubbie,Zxtoo,0.703619,dem,16
3,1Cubbie,edzsplace,0.699791,dem,16
...,...,...,...,...,...
1,zulran,cyndigist,1.000000,rep,24
2,zulran,dd9000,1.000000,rep,24
3,zulran,eliza_shines,1.000000,rep,24
4,zulran,islandinthemoon,1.000000,rep,24


In [17]:
path = "../data/"
df_graph.to_csv(path + "aristas_p_y.csv",index=False)